In [1]:
# upto 20190629
# merge email first
# then calculate decile
# Remove 18+
# Added the 18+, no quadrants

import pandas as pd
import numpy as np
import os
import datetime
import logging
import hashlib
import gc
import glob
logging.basicConfig(filename='BL_CRM_LR_Quarterly_upto20190629_2019Q3_'+str(datetime.datetime.now().date())+'.log', level=logging.INFO)
logging.info('Started')

samplerows = None

lastdate_date = datetime.date(2019,6,29) # Recent Saturday
active_Sunday = str(lastdate_date-datetime.timedelta(days=52*7-1))

lapsed_Sunday = str(lastdate_date-datetime.timedelta(days=52*7*1.5-1))
Beginning_18_months_ago=str(lastdate_date-datetime.timedelta(days=52*7*1.5-1))

lastdate=str(lastdate_date)
print("Lapsed Start on: "+lapsed_Sunday) #>=
print("Active Start on: "+active_Sunday) #>=
print("Store Allocation Starting on: "+Beginning_18_months_ago) #>=
print("Last Saturday: "+lastdate) #<=

def recursive_file_gen(root_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            yield os.path.join(root, file)
            
folder_write = '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q3/output_'+str(datetime.datetime.now().date())+'/'
try:
    os.stat(folder_write)
except:
    os.mkdir(folder_write)
    
os.getcwd()

Lapsed Start on: 2017-12-31
Active Start on: 2018-07-01
Store Allocation Starting on: 2017-12-31
Last Saturday: 2019-06-29


'/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q3'

In [2]:
chunksize_num = 10**7
filename='/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_0922/combinedtransactions_0922.csv'
dftrans_before_20180922=pd.DataFrame()
count_i=0

for chunk in pd.read_csv(filename, chunksize=chunksize_num,dtype=str,usecols=['customer_id_hashed','transaction_date','transaction_time',
                   'transaction_id','location_id','total_transaction_amt'],nrows=samplerows): #Add back the transaction info,,
    chunk['total_transaction_amt']=chunk['total_transaction_amt'].astype(float)
    chunk = chunk.drop_duplicates()
    
    dftrans_before_20180922=dftrans_before_20180922.append(chunk)
    count_i+=1
    print(count_i,datetime.datetime.now())


del chunk
print("Earliest Date:" + str(dftrans_before_20180922['transaction_date'].min()))
print("Latest Date:" + str(dftrans_before_20180922['transaction_date'].max()))
dftrans_before_20180922=dftrans_before_20180922.drop_duplicates()

logging.info("Deduped: "+str(datetime.datetime.now()))
del dftrans_before_20180922['transaction_time']

gc.collect()


1 2019-07-20 10:45:10.762833
2 2019-07-20 10:46:58.529379
3 2019-07-20 10:48:44.412085
4 2019-07-20 10:50:50.697999
5 2019-07-20 10:52:51.187061
6 2019-07-20 10:54:58.927663
7 2019-07-20 10:57:05.110811
8 2019-07-20 10:59:11.551565
9 2019-07-20 11:01:02.761387
10 2019-07-20 11:03:06.716849
11 2019-07-20 11:03:55.548122
12 2019-07-20 11:09:48.039840
13 2019-07-20 11:11:12.485388
14 2019-07-20 11:13:26.476073
15 2019-07-20 11:14:51.690950
Earliest Date:2016-06-26
Latest Date:2018-09-22


63

In [3]:
# Up to 2019-06-29
# All item level data, weekly and the 1-time transfered historical data
historical_daily_data_folder="/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/"
historical_daily_data_list=list(recursive_file_gen(historical_daily_data_folder))
historical_daily_data_list=[x for x in historical_daily_data_list if (".txt" in x) & ("DailySales" in x)]
historical_daily_df=pd.DataFrame({"file_path":historical_daily_data_list})
historical_daily_df['week_end_dt']=historical_daily_df['file_path'].apply(lambda x: x.split(".")[0].split("MediaStormDailySalesHistory")[1])
historical_daily_df['week_end_dt']=historical_daily_df['week_end_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y%m%d").date())
historical_daily_df=historical_daily_df[historical_daily_df['week_end_dt']<=datetime.date(2019,2,12)]
historical_daily_df=historical_daily_df[historical_daily_df['week_end_dt']>datetime.date(2018,9,22)]
print(historical_daily_df.shape)
print("Min_Date: "+str(historical_daily_df['week_end_dt'].min()))
print("Max_Date: "+str(historical_daily_df['week_end_dt'].max()))

(20, 2)
Min_Date: 2018-09-29
Max_Date: 2019-02-09


In [4]:
historical_daily_df['file_path'].nunique()

20

In [5]:
new_daily_data_folder="/home/jian/BigLots/2019_by_weeks/"
new_daily_data_list=list(recursive_file_gen(new_daily_data_folder))
new_daily_data_list=[x for x in new_daily_data_list if (".txt" in x) & ("DailySales" in x)]
new_daily_data_list=[x for x in new_daily_data_list if "hist" not in x]

new_daily_df=pd.DataFrame({"file_path":new_daily_data_list})

new_daily_df['week_end_dt']=new_daily_df['file_path'].apply(lambda x: x.split(".")[0].split("2019_by_weeks/MediaStorm_")[1][:10])
new_daily_df['week_end_dt']=new_daily_df['week_end_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
new_daily_df=new_daily_df[new_daily_df['week_end_dt']>historical_daily_df['week_end_dt'].max()]
new_daily_df=new_daily_df[new_daily_df['week_end_dt']<=lastdate_date]
print(new_daily_df.shape)
print("Min_Date: "+str(new_daily_df['week_end_dt'].min()))
print("Max_Date: "+str(new_daily_df['week_end_dt'].max()))

daily_df_file_after_20180922=historical_daily_df.append(new_daily_df)
new_dailysales_files=daily_df_file_after_20180922['file_path'].tolist()


(20, 2)
Min_Date: 2019-02-16
Max_Date: 2019-06-29


In [6]:
combined_rewards_transaction_after_20180922_agg=pd.DataFrame() 
count_i=1
print("Total Files: "+str(len(new_dailysales_files)))
for file_daily in new_dailysales_files:
    df=pd.read_table(file_daily,sep= '|',dtype =str,nrows=samplerows,
                     usecols=['customer_id_hashed','transaction_dt','transaction_id','location_id','item_transaction_amt'])
    df=df[~pd.isnull(df['customer_id_hashed'])]
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df=df.groupby(['customer_id_hashed','transaction_dt','transaction_id','location_id'])['item_transaction_amt'].sum().to_frame().reset_index()
    df=df.drop_duplicates()

    
    combined_rewards_transaction_after_20180922_agg=combined_rewards_transaction_after_20180922_agg.append(df)
    print(count_i,"done",datetime.datetime.now())
    count_i+=1
del df
gc.collect()
combined_rewards_transaction_after_20180922_agg=combined_rewards_transaction_after_20180922_agg.rename(columns={"transaction_dt":"transaction_date"})


Total Files: 40
1 done 2019-07-20 11:23:45.766165
2 done 2019-07-20 11:24:33.000708
3 done 2019-07-20 11:25:09.251107
4 done 2019-07-20 11:25:46.190610
5 done 2019-07-20 11:26:26.089717
6 done 2019-07-20 11:27:07.750756
7 done 2019-07-20 11:27:52.804896
8 done 2019-07-20 11:28:44.392066
9 done 2019-07-20 11:29:58.328503
10 done 2019-07-20 11:31:43.524216
11 done 2019-07-20 11:33:55.663935
12 done 2019-07-20 11:37:32.892162
13 done 2019-07-20 11:39:33.001181
14 done 2019-07-20 11:40:41.064506
15 done 2019-07-20 11:41:30.255223
16 done 2019-07-20 11:42:19.054430
17 done 2019-07-20 11:43:17.140180
18 done 2019-07-20 11:44:13.781181
19 done 2019-07-20 11:45:02.113582
20 done 2019-07-20 11:45:51.175755
21 done 2019-07-20 11:46:42.521128
22 done 2019-07-20 11:47:34.138530
23 done 2019-07-20 11:48:32.110443
24 done 2019-07-20 11:49:29.636375
25 done 2019-07-20 11:50:26.147599
26 done 2019-07-20 11:51:27.048758
27 done 2019-07-20 11:52:27.295986
28 done 2019-07-20 11:53:45.164824
29 done 2019-

In [7]:
gc.collect()

combined_rewards_transaction_after_20180922_agg=combined_rewards_transaction_after_20180922_agg.groupby(['customer_id_hashed','transaction_date','transaction_id','location_id'])['item_transaction_amt'].sum().to_frame().reset_index()
combined_rewards_transaction_after_20180922_agg=combined_rewards_transaction_after_20180922_agg.rename(columns={"item_transaction_amt":"total_transaction_amt"})

all_rewards_since_201606=dftrans_before_20180922.append(combined_rewards_transaction_after_20180922_agg)

del dftrans_before_20180922
del combined_rewards_transaction_after_20180922_agg
gc.collect()

42

In [8]:
# writing takes ~ 1 Hour, hashed to save time

# all_rewards_since_201606.to_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q3/BL_Rewards_Transactions_20160626_to_20190629.csv",index=False)
gc.collect()

0

In [9]:
all_rewards_since_201606.to_csv("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q3/BL_Rewards_Transactions_20160626_to_20190629.csv",index=False)


In [10]:
#Getting the store for an id

frequently_visit_stores_18_months=all_rewards_since_201606[all_rewards_since_201606['transaction_date']>=Beginning_18_months_ago]

frequently_visit_stores_2=frequently_visit_stores_18_months.groupby(['customer_id_hashed','location_id'])['total_transaction_amt'].sum().to_frame().reset_index().rename(columns={"total_transaction_amt":"sales"})
frequently_visit_stores_18_months=frequently_visit_stores_18_months.groupby(['customer_id_hashed','location_id'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans"})

frequently_visit_stores_18_months=pd.merge(frequently_visit_stores_18_months,frequently_visit_stores_2,on=['customer_id_hashed','location_id'],how="outer")
del frequently_visit_stores_2
print(frequently_visit_stores_18_months.shape)
frequently_visit_stores_18_months=frequently_visit_stores_18_months.sort_values(['customer_id_hashed','trans','sales'],ascending=[True,False,False])

frequently_visit_stores_18_months=frequently_visit_stores_18_months[['customer_id_hashed','location_id']].drop_duplicates("customer_id_hashed")
print(frequently_visit_stores_18_months.shape)
frequently_visit_stores_18_months.to_csv(folder_write+"frequently_visit_stores_18_months.csv",index=False)
del frequently_visit_stores_18_months
gc.collect()

(28365769, 4)
(20821581, 2)


7

In [11]:
del all_rewards_since_201606['transaction_id']
del all_rewards_since_201606['location_id']
gc.collect()

14

In [12]:
# Check the data here
print("all_rewards_since_201606.shape",all_rewards_since_201606.shape)
print("all_rewards_since_201606['customer_id_hashed'].nunique()",all_rewards_since_201606['customer_id_hashed'].nunique())
print("all_rewards_since_201606['transaction_date'].nunique()",all_rewards_since_201606['transaction_date'].nunique())
print("all_rewards_since_201606['transaction_date'].min()",all_rewards_since_201606['transaction_date'].min())
print("all_rewards_since_201606['transaction_date'].max()",all_rewards_since_201606['transaction_date'].max())
all_rewards_since_201606.head(2)

all_rewards_since_201606.shape (201292675, 3)
all_rewards_since_201606['customer_id_hashed'].nunique() 27232239
all_rewards_since_201606['transaction_date'].nunique() 1097
all_rewards_since_201606['transaction_date'].min() 2016-06-26
all_rewards_since_201606['transaction_date'].max() 2019-06-29


,customer_id_hashed,transaction_date,total_transaction_amt
0,bc45d2747297addd8bcad579da77756b007d27cd634c71...,2018-03-20,22.25
1,f627e56e080cb699864d9c488345aee82c98fa66ebc2f2...,2018-03-10,16.00


In [13]:
print(lastdate_date-datetime.date(2016,6,26))
print(365*4)
Beginning_18_months_ago

1098 days, 0:00:00
1460


'2017-12-31'

In [14]:
# Within 18 months only for the dataframe all_rewards_since_201606
df_18_plus_ids=all_rewards_since_201606[all_rewards_since_201606['transaction_date']<Beginning_18_months_ago]
print("check_point_1 "+str(datetime.datetime.now()))
all_rewards_since_201606=all_rewards_since_201606[all_rewards_since_201606['transaction_date']>=Beginning_18_months_ago]
print("check_point_2 "+str(datetime.datetime.now()))
gc.collect()
print("check_point_3 "+str(datetime.datetime.now()))

df_18_plus_ids=df_18_plus_ids[~df_18_plus_ids['customer_id_hashed'].isin(all_rewards_since_201606['customer_id_hashed'].unique().tolist())]
df_18_plus_ids=df_18_plus_ids[['customer_id_hashed']].drop_duplicates()
print("check_point_4 "+str(datetime.datetime.now()))


check_point_1 2019-07-20 13:52:42.787281
check_point_2 2019-07-20 13:53:11.252405
check_point_3 2019-07-20 13:53:24.697257
check_point_4 2019-07-20 14:11:33.059977


In [15]:
###get recency
dfrecency=all_rewards_since_201606[['customer_id_hashed','transaction_date']].sort_values("transaction_date",ascending=False).drop_duplicates()#Allready combined

print (min(dfrecency['transaction_date']))
print (max(dfrecency['transaction_date']))
dfrecency = dfrecency.drop_duplicates('customer_id_hashed')
dfrecency.to_csv(folder_write + 'dfrecency.csv',index = False)

2017-12-31
2019-06-29


In [16]:
dfrecency['transaction_date'] = pd.to_datetime(dfrecency['transaction_date'])
dfrecency['recency'] =  lastdate_date - dfrecency['transaction_date']
dfrecency['recency'] = dfrecency['recency'].apply(lambda x:x.days)
dfrecency['recency'] = np.ceil((dfrecency['recency']+1)/30)

dfrecency = dfrecency[['customer_id_hashed','recency']]
dfrecency = dfrecency.drop_duplicates('customer_id_hashed')
dfrecency.to_csv(folder_write + 'dfrecency2.csv',index = False)

dfrecency.shape

(20821581, 2)

In [17]:
all_rewards_since_201606['transactions'] = 1
dftotal = all_rewards_since_201606[['customer_id_hashed','total_transaction_amt','transactions']].groupby(['customer_id_hashed']).sum().reset_index().rename(columns={"total_transaction_amt":"sales"})

dftotal = pd.merge(dftotal,dfrecency,on = 'customer_id_hashed',how='outer')
del dfrecency
del all_rewards_since_201606
gc.collect()

67

In [18]:
dftotal = dftotal.sort_values(['transactions','recency','sales'],ascending = [0,1,0])
dftotal.reset_index(drop = True, inplace = True)
dftotal.reset_index(inplace = True)
dftotal = dftotal.rename(columns = {'index':'Transindex'})

dftotal = dftotal.sort_values(['sales','recency','transactions'],ascending = [0,1,0])
dftotal.reset_index(drop = True, inplace = True)
dftotal.reset_index(inplace = True)
dftotal = dftotal.rename(columns = {'index':'Amtindex'})

dftotal = dftotal.sort_values(['recency','transactions','sales'],ascending = [1,0,0])
dftotal.reset_index(drop = True, inplace = True)
dftotal.reset_index(inplace = True)
dftotal = dftotal.rename(columns = {'index':'recencyindex'})

c_ids = len(dftotal.index)
logging.info('total customers from transaction and amt: ')
logging.info(c_ids)
c_ids = np.ceil(c_ids/5.0)

dftotal['Transindex'] = np.ceil((dftotal['Transindex']+1)/c_ids)
dftotal['Amtindex'] = np.ceil((dftotal['Amtindex']+1)/c_ids)
dftotal['recencyindex'] = np.ceil((dftotal['recencyindex']+1)/c_ids)

dftotal['RFM'] = dftotal['recencyindex']*100 + dftotal['Transindex']*10 + dftotal['Amtindex']


dftotal = dftotal[['customer_id_hashed','RFM','recency','transactions','sales']]



In [19]:
dfiddetail = pd.read_csv('/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/combined_masterids_up_to_20181229_JL.csv',nrows = samplerows)
dfiddetail = dfiddetail.drop_duplicates('customer_id_hashed')
###
master_file_2018Q1_2019Q1=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/All Rewards Members 2018-02-04 - 2019-05-04.zip",
                                             compression="zip",dtype=str,sep="|",usecols=['customer_id_hashed','email_address_hash','customer_zip_code'])
dfiddetail=master_file_2018Q1_2019Q1.append(dfiddetail).drop_duplicates("customer_id_hashed")
del master_file_2018Q1_2019Q1
###
master_file_gap=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/MediaStorm Rewards Master P4 2019 - no transaction info.zip",
                                             compression="zip",dtype=str,sep="|",usecols=['customer_id_hashed','email_address_hash','customer_zip_code'])
dfiddetail=master_file_gap.append(dfiddetail).drop_duplicates("customer_id_hashed")
del master_file_gap
###

all_master_files_after_201906=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
all_master_files_after_201906=[x for x in all_master_files_after_201906 if ("aster" in x) & (".txt" in x)]
all_master_files_after_201906=[x for x in all_master_files_after_201906 if x.split("ts/2019_by_weeks/MediaStorm_")[1][:10]>"2019-06-01"]
df_master_all_later_after_201906=pd.DataFrame()
for file in all_master_files_after_201906:
    df=pd.read_table(file,dtype=str,sep="|",usecols=['customer_id_hashed','email_address_hash','customer_zip_code'])
    df_master_all_later_after_201906=df_master_all_later_after_201906.append(df)

dfiddetail=df_master_all_later_after_201906.append(dfiddetail).drop_duplicates("customer_id_hashed")
del df_master_all_later_after_201906

dfiddetail = dfiddetail.drop_duplicates('email_address_hash')

#dropped both email & id duplicated
###
gc.collect()
print("dfiddetail.shape",dfiddetail.shape)

dfiddetail.shape (28200021, 3)


In [20]:
print(datetime.datetime.now())

2019-07-20 14:45:54.025712


In [21]:
dfrecency = pd.read_csv(folder_write + 'dfrecency.csv')
dfrecency['active'] = np.where(dfrecency['transaction_date']>=active_Sunday,'active',
                               np.where(dfrecency['transaction_date']>=lapsed_Sunday,'lapsed','other')
                              )

print(dfrecency['active'].unique().tolist())

dftotal = pd.merge(dftotal,dfrecency[['customer_id_hashed','active']],on = 'customer_id_hashed')

logging.info(str(dfrecency['active'].unique().tolist()))
del dfrecency
gc.collect()

['active', 'lapsed']


57

In [22]:
dfiddetail['customer_zip_code'] = dfiddetail['customer_zip_code'].astype('str')
dfiddetail['customer_zip_code'] = dfiddetail['customer_zip_code'].str[0:5]
dfiddetail['customer_zip_code'].fillna('00000',inplace = True)
dfiddetail['customer_zip_code'] = dfiddetail['customer_zip_code'].apply(lambda x:x.zfill(5))
print(len(dfiddetail.index))


print("totalids_trans:",len(dftotal.index))
dftotal = pd.merge(dftotal,dfiddetail,on = 'customer_id_hashed')
print("totalids_trans_mergewithmaster:",len(dftotal.index))

del dfiddetail['customer_zip_code']
gc.collect()

28200021
totalids_trans: 20821581
totalids_trans_mergewithmaster: 18936098


21

In [23]:
lapsed_trans=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/lapsed20140826_20170226/MediaStormLapsedCustDtl.txt",
                     sep=",",usecols=['customer_id_hashed'],dtype=str).drop_duplicates() # Doesn't go to score at all, so no need to read all columns

lapsed_master=pd.read_csv('/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/MediaStorm_Lapsed_Reward_Member_Master_from2014-08-26to2017-02-26.zip',
                     nrows = samplerows,dtype = 'str',sep = '|',
                       usecols = ['customer_id_hashed','email_address_hash','customer_zip_code'])

lapsed_master=lapsed_master.drop_duplicates("customer_id_hashed")
print(lapsed_master.shape)

lapsed_master=pd.merge(lapsed_master,lapsed_trans,on="customer_id_hashed",how="outer")
print(lapsed_master.shape)
lapsed_master=lapsed_master[~pd.isnull(lapsed_master['email_address_hash'])]
print(lapsed_master.shape)

lapsed_master=lapsed_master[~lapsed_master['customer_id_hashed'].isin(dftotal['customer_id_hashed'].tolist())]
print(lapsed_master.shape)

(1873409, 3)
(1873954, 3)
(1873409, 3)
(1352150, 3)


In [24]:
df_18_plus_ids=pd.merge(df_18_plus_ids,dfiddetail,on = 'customer_id_hashed')

df_18_plus_ids=df_18_plus_ids.append(lapsed_master)
df_18_plus_ids=df_18_plus_ids.drop_duplicates("customer_id_hashed").drop_duplicates("email_address_hash")
df_18_plus_ids.to_csv(folder_write+"df_ids_email_18_plus_shoppers.csv",index=False)

del dfiddetail
gc.collect()


70

In [25]:
dftotal = dftotal.sort_values(['RFM','recency','transactions',
                               'sales'],ascending = [1,1,0,0])
dftotal.reset_index(drop = True, inplace = True)
dftotal.reset_index(inplace = True)
dftotal = dftotal.rename(columns = {'index':'frmindex'})
c_ids = len(dftotal.index)
c_ids = np.ceil(c_ids/10.0)
dftotal['frmindex'] = np.ceil((dftotal['frmindex']+1)/c_ids)

print("Done with the RFM scoring, "+str(datetime.datetime.now()))

Done with the RFM scoring, 2019-07-20 14:59:53.383055


# Getting the primary stores for each member

In [26]:
frequently_visit_stores_18_months=pd.read_csv(folder_write+"frequently_visit_stores_18_months.csv",dtype=str)
register_stores=pd.read_csv("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/output_sing_up_location/BL_id_by_register_store_JL_2019-07-12.csv",
                            dtype=str,nrows=samplerows,usecols=['customer_id_hashed','sign_up_location'])
register_stores=register_stores.rename(columns={"sign_up_location":"location_id"})


In [27]:
store_for_ids=frequently_visit_stores_18_months.append(register_stores)
store_for_ids=store_for_ids.drop_duplicates("customer_id_hashed")
store_for_ids.shape

(30668013, 2)

In [28]:
del frequently_visit_stores_18_months
del register_stores
gc.collect()

21

In [29]:
dftotal=pd.merge(dftotal,store_for_ids,on="customer_id_hashed",how="left")


In [30]:
del store_for_ids
gc.collect()

21

In [31]:
# read the quadrant by store for 2019 Q1

Q1_2019_store_quadrant=pd.read_excel("/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q3/Excel_BL_2019_Q1_post_quadrants_JL_2019-05-15.xlsx",
                                dtype=str,sheetname="2019_Q1_Store_Quad_Defination",usecols=['location_id','Quadrant'])
print(Q1_2019_store_quadrant.shape)
print(len(Q1_2019_store_quadrant['location_id'].unique()))

dftotal=pd.merge(dftotal,Q1_2019_store_quadrant,on="location_id",how="left")
dftotal['Quadrant']=dftotal['Quadrant'].fillna("Quadrant III")


(1355, 2)
1355


In [32]:
dftotal['frmindex']=dftotal['frmindex'].apply(lambda x: str(int(float(x))).zfill(2))
dftotal['customer_zip_code']=dftotal['customer_zip_code'].apply(lambda x: x.zfill(5))
dftotal['frmindex']=dftotal['frmindex'].apply(lambda x:"D"+x)

# zip_PST

In [33]:
zipmap=pd.read_excel('/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx',
                     dtype = 'str',sheetname="unique_zips_full_footprint")
zipmap=zipmap.rename(columns={"zip_type":"zipcodegroup","zip_cd":"customer_zip_code"})

zipmap['zipcodegroup']=zipmap['zipcodegroup'].replace("trans_P","P")
zipmap['zipcodegroup']=zipmap['zipcodegroup'].replace("trans_S","S")
zipmap['zipcodegroup']=zipmap['zipcodegroup'].replace("zips_10","S")


In [34]:
dftotal = pd.merge(dftotal,zipmap,on ='customer_zip_code',how = 'left' )
print(dftotal['zipcodegroup'].unique())
dftotal['zipcodegroup'].fillna('T',inplace = True)
print(dftotal['zipcodegroup'].unique())

dftotal.to_csv(folder_write + 'dfrfm_wemail.csv',index = False)
print("Final wemailcsv:",dftotal.shape)

['P' nan 'S']
['P' 'T' 'S']
Final wemailcsv: (18936098, 12)


In [35]:
dftotal.shape

(18936098, 12)

# Summary & Output

In [36]:
df_H=pd.DataFrame({"frmindex":['D01','D02','D03','D04']})
df_H['HML_Group']="H"

df_M=pd.DataFrame({"frmindex":['D05','D06','D07']})
df_M['HML_Group']="M"

df_L=pd.DataFrame({"frmindex":['D08','D09','D10']})
df_L['HML_Group']="L"

df_HML=df_H.append(df_M).append(df_L)
gc.collect()

38377

In [37]:
dftotal=pd.merge(dftotal,df_HML,on='frmindex',how="left")


dftotal['segment_Decile']=dftotal['Quadrant']+"_"+dftotal['zipcodegroup']+"_"+dftotal['frmindex']+"_2019Q3"
dftotal['segment_2019Q3']=dftotal['Quadrant']+"_"+dftotal['zipcodegroup']+"_"+dftotal['HML_Group']+"_2019Q3"

dftotal.to_csv(folder_write + 'dfrfm_final_details_wemail_ziplabel_StoreQuad.csv',index = False)
gc.collect()

28